# SLEAP Distance Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys
from collections import defaultdict

In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import sleap.process_pose

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

# Functions

In [8]:
def combine_dicts(dicts):
    """
    Combine lists from multiple dictionaries that share the same key.

    This function takes a list of dictionaries where each dictionary's values are lists of numbers.
    It merges these lists for each corresponding key across all dictionaries, producing a single
    dictionary where each key has a combined list of all numbers from the input dictionaries.

    Parameters:
        dicts (list of dict): A list of dictionaries with values as lists of numbers.

    Returns:
        dict: A dictionary with keys from the input dictionaries and values as merged lists 
        of numbers from all corresponding input dictionary values.
    """
    combined = defaultdict(list)
    for dictionary in dicts:
        for key, value in dictionary.items():
            combined[key].extend(value)
    
    return dict(combined)


# Example usage
list_of_dicts = [
    {'a': [1, 2], 'b': [3, 4]},
    {'a': [5], 'b': [6, 7]},
    {'a': [8, 9], 'c': [10]}
]

combined_dict = combine_dicts(list_of_dicts)
print(combined_dict)

{'a': [1, 2, 5, 8, 9], 'b': [3, 4, 6, 7], 'c': [10]}


In [9]:
def find_consecutive_ranges(numbers, min_length=1):
    """
    Finds the start and end indices for consecutive ranges of each number in a list where the range meets a minimum length.

    This function iterates through a list of integers and identifies ranges where the same integer appears consecutively
    and the length of this sequence meets or exceeds the specified minimum length.

    Parameters:
        numbers (list): A list of integers to analyze for consecutive ranges.
        min_length (int): The minimum length of a range for it to be included in the results.

    Returns:
        dict: A dictionary with integers as keys and a list of tuples (start, end) as values,
              where each tuple represents the start and end indices (inclusive) of consecutive ranges
              for that integer. Only ranges that meet or exceed the minimum length are included.
    """
    ranges = {}
    n = len(numbers)
    if n == 0:
        return ranges
    
    start = 0
    current = numbers[0]

    for i in range(1, n):
        if numbers[i] != current:
            if (i - start) >= min_length:
                if current not in ranges:
                    ranges[current] = []
                ranges[current].append((start, i - 1))
            current = numbers[i]
            start = i

    # Handle the last range
    if (n - start) >= min_length:
        if current not in ranges:
            ranges[current] = []
        ranges[current].append((start, n - 1))

    return ranges

# Example usage:
numbers = [1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1]
print(find_consecutive_ranges(numbers, min_length=3))

{2: [(2, 4)], 3: [(5, 8)]}


In [10]:
def update_tuples_in_dict(original_dict, reference_list):
    """
    Updates the values in the tuples within a dictionary by replacing indices with corresponding values from a reference list.
    
    This function iterates through each key-value pair in the original dictionary. Each value is expected to be a list of tuples,
    where each tuple contains indices. These indices are used to fetch corresponding values from the reference list, creating new tuples.
    
    Parameters:
        original_dict (dict): Dictionary whose values are lists of tuples. Each tuple consists of indices into the reference_list.
        reference_list (list): List of elements that are referenced by the indices in the tuples of the original_dict.
        
    Returns:
        dict: A dictionary with the same keys as original_dict but with tuples transformed to contain elements from reference_list
              based on the indices in the original tuples.
    """
    # Create a new dictionary to store the updated key-value pairs
    new_dict = {}
    for key, list_of_tuples in original_dict.items():
        # Process each tuple in the list associated with the current key
        updated_tuples = [
            tuple(reference_list[idx] for idx in tup) for tup in list_of_tuples
        ]
        new_dict[key] = updated_tuples
    
    return new_dict

# Example usage:
original_dict = {
    'a': [(0, 1), (2, 3)],
    'b': [(1, 3), (0, 2)]
}
reference_list = ['alpha', 'beta', 'gamma', 'delta']

updated_dict = update_tuples_in_dict(original_dict, reference_list)
print(updated_dict)

{'a': [('alpha', 'beta'), ('gamma', 'delta')], 'b': [('beta', 'delta'), ('alpha', 'gamma')]}


## Inputs & Data

Explanation of each input and where it comes from.

In [11]:
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [12]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_01_lfp_traces_and_frames.pkl".format(OUTPUT_PREFIX))



In [13]:
# SLEAP_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")
SLEAP_DF = pd.read_pickle("../2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")

In [14]:
SLEAP_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,subject_nose,subject_tail_base,agent_thorax,agent_nose,agent_tail_base,session_dir,experiment,sleap_name,video_id,agent
0,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[925, 926, 927, 928, 929, 930, 931, 932, 933, ...","[1125, 1125, 1125, 1125, 1125, 1125, 1125, 112...","[1324, 1324, 1324, 1324, 1324, 1324, 1324, 132...","[1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[Subj 1 blocking Subj 2, Subj 1 blocking Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[925, 925, 925, 925, 925, 925, 925, 925, 925, ...","[1524, 1524, 1524, 1524, 1524, 1524, 1524, 152...",...,"[[45.247421563585775, 36.62801880316424], [45....","[[54.22440917025702, 31.655120167227278], [54....","[[56.22540529334203, 16.13440966384005], [56.2...","[[63.74425299761245, 16.652677097245252], [62....","[[53.01324097350247, 14.411009115644015], [53....",20230612_112630_standard_comp_to_training_D1_s...,standard,20230612_112630_standard_comp_to_training_D1_s...,23,1.2
1,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[925, 926, 927, 928, 929, 930, 931, 932, 933, ...","[1125, 1125, 1125, 1125, 1125, 1125, 1125, 112...","[1324, 1324, 1324, 1324, 1324, 1324, 1324, 132...","[1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[Subj 1 blocking Subj 2, Subj 1 blocking Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[925, 925, 925, 925, 925, 925, 925, 925, 925, ...","[1524, 1524, 1524, 1524, 1524, 1524, 1524, 152...",...,"[[63.74425299761245, 16.652677097245252], [62....","[[53.01324097350247, 14.411009115644015], [53....","[[50.98879914736345, 32.938384713243906], [50....","[[45.247421563585775, 36.62801880316424], [45....","[[54.22440917025702, 31.655120167227278], [54....",20230612_112630_standard_comp_to_training_D1_s...,standard,20230612_112630_standard_comp_to_training_D1_s...,23,1.1
2,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[716, 717, 718, 719, 720, 721, 722, 723, 724, ...","[916, 916, 916, 916, 916, 916, 916, 916, 916, ...","[1117, 1117, 1117, 1117, 1117, 1117, 1117, 111...","[1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, ...","[Subj 2 Only, Subj 2 Only, Subj 2 Only, Subj 2...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[716, 716, 716, 716, 716, 716, 716, 716, 716, ...","[1317, 1317, 1317, 1317, 1317, 1317, 1317, 131...",...,"[[60.51278680136493, 28.85765727660519], [60.5...","[[51.37353954574669, 29.98946836796922], [51.3...","[[34.33527733807332, 19.286478831937885], [34....","[[26.459160293275204, 20.924631150658147], [26...","[[37.78645593948065, 18.012081792725134], [37....",20230613_105657_standard_comp_to_training_D2_s...,standard,20230613_105657_standard_comp_to_training_D2_s...,24,1.4
3,20230613_105657_standard_comp_to_training_D2_s...,1.4,"[716, 717, 718, 719, 720, 721, 722, 723, 724, ...","[916, 916, 916, 916, 916, 916, 916, 916, 916, ...","[1117, 1117, 1117, 1117, 1117, 1117, 1117, 111...","[1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, 1.4, ...","[Subj 2 Only, Subj 2 Only, Subj 2 Only, Subj 2...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[716, 716, 716, 716, 716, 716, 716, 716, 716, ...","[1317, 1317, 1317, 1317, 1317, 1317, 1317, 131...",...,"[[26.459160293275204, 20.924631150658147], [26...","[[37.78645593948065, 18.012081792725134], [37....","[[55.58212106205608, 30.63306686571954], [55.7...","[[60.51278680136493, 28.85765727660519], [60.5...","[[51.37353954574669, 29.98946836796922], [51.3...",20230613_105657_standard_comp_to_training_D2_s...,standard,20230613_105657_standard_comp_to_training_D2_s...,24,1.1
4,20230614_114041_standard_comp_to_training_D3_s...,1.1,"[655, 656, 657, 658, 659, 660, 661, 662, 663, ...","[855, 855, 855, 855, 855, 855, 855, 855, 855, ...","[1055, 1055, 1055, 1055, 1055, 1055, 1055, 105...","[1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, ...","[Subj 2 blocking Subj 1, Subj 2 blocking Subj ...

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
FULL_LFP_TRACES_PKL = "{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

## Merging the dataframes based on shared video name

In [17]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [18]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_RMS_filtered_power_theta,vHPC_RMS_filtered_power_gamma,BLA_RMS_filtered_power_theta,BLA_RMS_filtered_power_gamma,LH_RMS_filtered_power_theta,LH_RMS_filtered_power_gamma,MD_RMS_filtered_power_theta,MD_RMS_filtered_power_gamma,mPFC_RMS_filtered_power_theta,mPFC_RMS_filtered_power_gamma
0,NaN,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[0.014285023, 0.022181146, 0.044097915, 0.0591...","[0.0017202137, 0.0012658514, 0.0011930467, 0.0...","[0.015456602, 0.025747435, 0.034084156, 0.0321...","[0.0023020238, 0.0016523497, 0.0015088222, 0.0...","[0.016193103, 0.025915565, 0.03433929, 0.03140...","[0.0018237891, 0.001400824, 0.001404246, 0.001...","[0.01444151, 0.02424808, 0.031674195, 0.028802...","[0.0024518063, 0.0018558174, 0.0014195474, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,NaN,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[0.014285023, 0.022181146, 0.044097915, 0.0591...","[0.0017202137, 0.0012658514, 0.0011930467, 0.0...","[0.015456602, 0.025747435, 0.034084156, 0.0321...","[0.0023020238, 0.0016523497, 0.0015088222, 0.0...","[0.016193103, 0.025915565, 0.03433929, 0.03140...","[0.0018237891, 0.001400824, 0.001404246, 0.001...","[0.01444151, 0.02424808, 0.031674195, 0.028802...","[0.0024518063, 0.0018558174, 0.0014195474, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
2,NaN,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.021758892, 0.030758785, 0.024464468, nan, n...","[0.0007876017, 0.0012533057, 0.0011539572, nan...","[0.022868399, 0.03210648, 0.025050245, nan, na...","[0.0007347132, 0.0011532403, 0.0011211544, nan...","[0.022456143, 0.031618252, 0.025473533, nan, n...","[0.00077316794, 0.0012196691, 0.0011400171, na...","[0.023196202, 0.032333095, 0.025302932, nan, n...","[0.0007058642, 0.0011378538, 0.001115803, nan,...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
3,NaN,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.021758892, 0.030758785, 0.024464468, nan, n...","[0.0007876017, 0.0012533057, 0.0011539572, nan...","[0.022868399, 0.03210648, 0.025050245, nan, na...","[0.0007347132, 0.0011532403, 0.0011211544, nan...","[0.022456143, 0.031618252, 0.025473533, nan, n...","[0.00077316794, 0.0012196691, 0.0011400171, na...","[0.023196202, 0.032333095, 0.025302932, nan, n...","[0.0007058642, 0.0011378538, 0.001115803, nan,...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,NaN,20240320_171038_alone_comp_subj_4-2_

In [19]:
SLEAP_DF["video_name"] = SLEAP_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [20]:
cols_to_use = LFP_SPECTRAL_DF.columns.difference(SLEAP_DF.columns)
merged_LFP_SPECTRAL_DF = pd.merge(SLEAP_DF, LFP_SPECTRAL_DF[list(cols_to_use) + ["video_name", "current_subject"]], on=["video_name", "current_subject"], how="inner")

In [21]:
merged_LFP_SPECTRAL_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,vHPC_lfp_RMS,vHPC_lfp_RMS_filtered,vHPC_lfp_mask,vHPC_lfp_modified_zscore,vHPC_lfp_trace,vHPC_lfp_trace_filtered,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[-0.07440248, -0.6211742, -0.88417834, -0.8824...","[-0.079, -0.661, -0.941, -0.939, -1.488, -1.78...","[False, False, False, False, False, False, Fal...","[-0.066892564, -0.65406066, -0.93649584, -0.93...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[-0.0071436157, -0.03571808, -0.47326455, -0.7...","[-0.007, -0.037, -0.495, -0.804, -0.191, 0.058...","[False, False, False, False, False, False, Fal...","[-0.0018840783, -0.03202933, -0.4936285, -0.80...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831..."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.3,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[-0.3585364, 0.2145572, 0.35289013, -0.3585364...","[-0.413, 0.247, 0.407, -0.413, -0.615, -0.218,...","[False, False, False, False, False, False, Fal...","[-0.44384193, 0.26560622, 0.43685234, -0.44384...","[-24.765, 14.82, 24.375, -24.765, -36.855, -13...","[-24.765, 14.82, 24.375, -24.765, -36.855, -13...","[[0.014051933, 0.6209912, 0.20073363, 0.871971...","[0.069354706, 0.071877405, 0.07377358, 0.07502...","[-0.8432274, -0.42268786, -0.36932397, -0.1285...","[1384, 1983, 2770, 4156, 5541, 5541, 6927, 831..."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.4,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[-0.46712062, -0.873221, 0.016833175, 0.831138...","[-0.469, -0.878, 0.017, 0.835, -0.533, -0.996,...","[False, False, False, False, False, False, Fal...","[-0.46027198, -0.8696384, 0.

## Getting the timestamps of each cluster frame

In [24]:
merged_LFP_SPECTRAL_DF["frame_index"].iloc[0]

array([34042, 34043, 34044, ..., 61590, 61591, 61592])

In [25]:
merged_LFP_SPECTRAL_DF["video_timestamps"].iloc[0]

array([    1384,     2770,     4156, ..., 64384726, 64384726, 64386112],
      dtype=int32)

In [26]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"] = merged_LFP_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["frame_index"]], axis=1)


In [27]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].head()

0    [34079916, 34081302, 34082688, 34084074, 34084...
1    [34079916, 34081302, 34082688, 34084074, 34084...
2    [34103476, 34104862, 34106248, 34106248, 34107...
3    [34103476, 34104862, 34106248, 34106248, 34107...
4    [34096547, 34097933, 34098977, 34099318, 34100...
Name: cluster_timestamp, dtype: object

- Checking if the cluster timestamp shape matches the actual number of clusters

In [28]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].apply(lambda x: x.shape).head()

0    (12593,)
1    (12593,)
2    (12591,)
3    (12591,)
4    (12593,)
Name: cluster_timestamp, dtype: object

In [29]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].head()

0    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, ...
1    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, ...
2    [3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...
3    [3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...
4    [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 3, 2, 7, 7, ...
Name: kmeans_cluster, dtype: object

In [30]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].apply(lambda x: x.shape).head()

0    (12593,)
1    (12593,)
2    (12591,)
3    (12591,)
4    (12593,)
Name: kmeans_cluster, dtype: object

In [31]:
FULL_LFP_TRACES_PKL

'rce_pilot_3_alone_comp_04_spectral_and_sleap.pkl'

In [32]:
merged_LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))


# Finding the ranges of each cluster

In [35]:
for col in merged_LFP_SPECTRAL_DF.columns:
    print(col)

video_name
current_subject
frame_index
tone_start_frame
tone_stop_frame
condition 
competition_closeness
notes
10s_before_tone_frame
10s_after_tone_frame
subject_thorax_to_agent_thorax
nose_to_reward_port_sum
nose_to_reward_port_diff
thorax_velocity_sum
thorax_velocity_diff
to_reward_port_angle_sum
to_reward_port_angle_diff
subject_nose_to_reward_port
subject_thorax_velocity
subject_to_reward_port_angle
agent_nose_to_reward_port
agent_thorax_velocity
agent_to_reward_port_angle
closebool_subject_nose_to_reward_port
closebool_agent_nose_to_reward_port
movingbool_subject_thorax_velocity
movingbool_agent_thorax_velocity
manual_cluster_id
standard_embedding_x
standard_embedding_y
kmeans_cluster
subject_locations
agent_locations
subject_thorax
subject_nose
subject_tail_base
agent_thorax
agent_nose
agent_tail_base
session_dir
experiment
sleap_name
video_id
agent
BLA-to-LH_granger_gamma
BLA-to-LH_granger_theta
BLA-to-MD_granger_gamma
BLA-to-MD_granger_theta
BLA-to-mPFC_granger_gamma
BLA-to-mPF

## Separating each frame into its own row

In [66]:
filter_columns = ['video_name',
 'current_subject',
 'session_dir',
 'sleap_name',
 'video_id',
 'agent',
 'cluster_timestamp',
 'first_timestamp',
 'recording']

In [67]:
explode_columns = ['frame_index',
 'tone_start_frame',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'notes',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [68]:
# selecting columns to simplify
filtered_LFP_SPECTRAL_DF = merged_LFP_SPECTRAL_DF[filter_columns + explode_columns].copy()

In [69]:
filtered_LFP_SPECTRAL_DF.head()

,video_name,current_subject,session_dir,sleap_name,video_id,agent,cluster_timestamp,first_timestamp,recording,frame_index,...,standard_embedding_y,kmeans_cluster,subject_locations,agent_locations,subject_thorax,subject_nose,subject_tail_base,agent_thorax,agent_nose,agent_tail_base
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,"[34042, 34043, 34044, 34045, 34046, 34047, 340...",...,"[3.190685510635376, 3.1923599243164062, 3.1389...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, ...","[[[61.39644296346538, 31.82081289424359], [58....","[[[56.40002042465876, 33.47263135770957], [55....","[[54.225662928143386, 30.269937128249303], [54...","[[62.13302320168461, 34.13495024512682], [61.3...","[[50.83958858240558, 27.427984293098355], [50....","[[51.35546426976598, 33.93773101458849], [52.3...","[[60.438020774390075, 35.57292836881663], [60....","[[48.80137642581422, 31.16922356981376], [49.0..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.2,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-3_t5b5_merged,"[34042, 34043, 34044, 34045, 34046, 34047, 340...",...,"[3.190685510635376, 3.1923599243164062, 3.1389...","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, ...","[[[56.40002042465876, 33.47263135770957], [55....","[[[61.39644296346538, 31.82081289424359], [58....","[[51.35546426976598, 33.93773101458849], [52.3...","[[60.438020774390075, 35.57292836881663], [60....","[[48.80137642581422, 31.16922356981376], [49.0...","[[54.225662928143386, 30.269937128249303], [54...","[[62.13302320168461, 34.13495024512682], [61.3...","[[50.83958858240558, 27.427984293098355], [50...."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.3,20240322_120625_alone_comp_subj_3-3_and_3-4,20240322_120625_alone_comp_subj_3-3_and_3-4.1....,18,3.4,"[34103476, 34104862, 34106248, 34106248, 34107...",3618506,20240322_120625_alone_comp_subj_3-3_t6b6_merged,"[34069, 34070, 34071, 34072, 34073, 34074, 340...",...,"[4.082400798797607, 3.8915390968322754, 3.5150...","[3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[[[62.39121557118253, 33.65381153049498], [59....","[[[61.16399490425991, 32.70045972585698], [59....","[[55.04801537397536, 30.000063064242042], [54....","[[61.44525963789897, 35.79427285418545], [58.4...","[[55.16590132823613, 27.57204904320481], [55.1...","[[63.14227571637791, 29.264671717012554], [62....","[[61.53968210480167, 35.796468985578834], [61....","[[63.39944559401088, 26.348095697771708], [63...."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.4,20240322_120625_alone_comp_subj_3-3_and_3-4,20240322_120625_alone_comp_subj_3-3_and_3-4.1....,18,3.3,"[34103476, 34104862, 34106248, 34106248, 34107...",3618506,20240322_120625_alone_comp_subj_3-4_t5b5_merged,"[34069, 34070, 34071, 34072, 34073, 34074, 340...",...,"[4.082400798797607, 3.8915390968322754, 3.5150...","[3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[[[61.16399490425991, 32.70045972585698], [59....","[[[62.39121557118253, 33.65381153049498], [59....","[[63.14227571637791, 29.264671717012554], [62....","[[61.53968210480167, 35.796468985578834], [61....","[[63.39944559401088, 26.348095697771708], [63....","[[55.04801537397536, 30.000063064242042], [54....","[[61.44525963789897, 35.79427285418545], [58.4...","[[55.16590132823613, 27.57204904320481], [55.1..."
4,20240322_160946_alone_comp_subj_4-3_and_4-4.1,4.3,20240322_160946_alone_comp_subj_4-3_and_4-4,20240322_160946_alone_comp_subj_4-3_and_4-4.1....,19,4.4,"[34096547, 34097933, 34098977, 34099318, 34100...",5331441,20240322_160946_alone_comp_subj_4-3_t6b6_merged,"[34063, 34064, 34065, 34066, 34067, 34068, 340...",...,"[2.9100699424743652, 2.952531099319458, 3.0866...","

In [70]:
# explode_columns = [
#     "cluster_timestamp",
#     "kmeans_cluster",
#     "condition ",
#     "competition_closeness",
#     "tone_start_frame"]

In [71]:
exploded_LFP_SPECTRAL_DF = filtered_LFP_SPECTRAL_DF.explode(explode_columns).reset_index(drop=True)


In [72]:
exploded_LFP_SPECTRAL_DF.head()

,video_name,current_subject,session_dir,sleap_name,video_id,agent,cluster_timestamp,first_timestamp,recording,frame_index,...,standard_embedding_y,kmeans_cluster,subject_locations,agent_locations,subject_thorax,subject_nose,subject_tail_base,agent_thorax,agent_nose,agent_tail_base
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34042,...,3.190686,7,"[[61.39644296346538, 31.82081289424359], [58.0...","[[56.40002042465876, 33.47263135770957], [55.8...","[54.225662928143386, 30.269937128249303]","[62.13302320168461, 34.13495024512682]","[50.83958858240558, 27.427984293098355]","[51.35546426976598, 33.93773101458849]","[60.438020774390075, 35.57292836881663]","[48.80137642581422, 31.16922356981376]"
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34043,...,3.19236,7,"[[60.893447251263716, 31.657960394179256], [57...","[[57.38510672506201, 33.769920948338815], [56....","[54.093177424666216, 30.357495891388663]","[61.380101836922606, 33.87305102712287]","[50.82106479966556, 27.597267245910636]","[52.330529932218774, 33.8102838308305]","[60.92987860509969, 35.874839292807806]","[49.09809097091966, 31.32314908841624]"
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34044,...,3.138943,7,"[[59.87908225928568, 30.84302729617087], [57.3...","[[57.87513895678869, 34.08114552153877], [57.1...","[53.10429595161392, 30.011529127728558]","[60.62718047216061, 33.61115180911892]","[50.812633642217584, 27.267762814877297]","[52.83099744898351, 33.94467235443203]","[60.63046309493748, 35.73704734584919]","[49.57547070748706, 31.33838790715778]"
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34045,...,3.118031,7,"[[58.36202102244704, 30.354798556960766], [55....","[[58.10863709799633, 34.13438612116752], [57.3...","[52.34659750686347, 29.552332224860816]","[59.37134114152587, 32.98493311917642]","[50.341439058040294, 26.935080983662623]","[53.07077864536584, 33.95712791257968]","[60.39906698410451, 35.89885752416092]","[49.80790343603905, 31.50044652416681]"
4,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,20240320_171038_alone_comp_subj_4-2_and_4-3,20240320_171038_alone_comp_subj_4-2_and_4-3.1....,17,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34046,...,3.124077,7,"[[57.40112822780809, 29.879193492310034], [54....","[[57.849880038368745, 34.09347779431784], [57....","[51.84014064327776, 29.033743529655833]","[57.350604631980865, 32.966145176187645]","[50.09887627077862, 26.293457226526993]","[52.83703862669043, 33.937962641644624]","[59.884472671424554, 35.58214488409866]","[49.84497979645574, 31.502299540615894]"


# Grouping columns by shared video, subject, and tone start(aka grouping by trial)

In [73]:
explode_columns

['frame_index',
 'tone_start_frame',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'notes',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [74]:
groupby_columns = ["video_name", "current_subject", "tone_start_frame"]

In [75]:
filter_columns += ["tone_stop_frame",
"condition ",
"competition_closeness",
"notes",
"10s_before_tone_frame",
"10s_after_tone_frame"]

In [76]:
# Define aggregation dictionary
agg_dict = {col: list for col in explode_columns if col not in groupby_columns and col not in filter_columns}

agg_dict.update({col: 'first' for col in filter_columns if col not in groupby_columns})

# Apply groupby and aggregation
grouped_exploded_columns = exploded_LFP_SPECTRAL_DF.groupby(["video_name", "current_subject", "tone_start_frame"]).agg(agg_dict).reset_index()


In [77]:
grouped_exploded_columns.head()

,video_name,current_subject,tone_start_frame,frame_index,subject_thorax_to_agent_thorax,nose_to_reward_port_sum,nose_to_reward_port_diff,thorax_velocity_sum,thorax_velocity_diff,to_reward_port_angle_sum,...,agent,cluster_timestamp,first_timestamp,recording,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,34242,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[4.873386, 4.9368286, 4.95431, 4.9935803, 5.11...","[3.630166, 3.7930658, 4.140675, 4.730201, 5.42...","[1.4684393, 1.92691, 2.386264, 3.032248, 3.868...","[15.642463, 15.843375, 16.048643, 16.248707, 1...","[5.4306073, 5.6331806, 5.7147374, 5.6083183, 5...","[5.032282, 4.8800473, 4.662217, 4.401958, 4.11...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34442,4.3,After trial,nan,34042,34642
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,35241,"[35041, 35042, 35043, 35044, 35045, 35046, 350...","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[26.412424, 25.048779, 23.673517, 22.475124, 2...","[24.234488, 22.897377, 21.647272, 20.52266, 19...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,35439,4.3,Subj 2 Only,nan,35041,35639
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,36339,"[36139, 36140, 36141, 36142, 36143, 36144, 361...","[13.881315, 14.176304, 14.438505, 14.65004, 14...","[20.872814, 21.642649, 22.377028, 22.993326, 2...","[19.293379, 20.148054, 20.855904, 21.357841, 2...","[6.936893, 7.039064, 7.0063467, 6.898076, 6.69...","[5.934906, 6.0408955, 5.9854646, 5.8209095, 5....","[0.8359046, 0.79514295, 0.75987095, 0.7393834,...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,36538,4.3,Subj 2 blocking Subj 1,nan,36139,36738
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,37336,"[37136, 37137, 37138, 37139, 37140, 37141, 371...","[14.58712, 14.6317215, 14.698479, 14.7541, 14....","[4.5502543, 4.3937583, 4.5302463, 4.6787696, 4...","[2.9129052, 2.954475, 2.96101, 3.1235678, 3.26...","[4.4460034, 3.5049973, 2.642458, 1.9523432, 1....","[2.4809754, 1.6541765, 0.9319356, 0.37681082, ...","[3.5158348, 3.5706627, 3.5573924, 3.5392659, 3...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,37535,4.3,Subj 2 blocking Subj 1,nan,37136,37735
4,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,38834,"[38634, 38635, 38636, 38637, 38638, 38639, 386...","[16.687664, 16.676865, 16.725145, 16.772022, 1...","[22.407978, 22.735888, 22.8885, 22.941162, 22....","[20.59827, 20.763409, 20.856441, 20.875114, 20...","[11.273786, 9.234783, 7.4643965, 6.0377355, 5....","[9.619022, 7.865918, 6.3134904, 5.0081577, 4.0...","[1.2626256, 1.244121, 1.215351, 1.2018311, 1.1...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,39033,4.3,Subj 2 Only,nan,38634,39233


## Renaming the trial labels

In [79]:
grouped_exploded_columns.head()

,video_name,current_subject,tone_start_frame,frame_index,subject_thorax_to_agent_thorax,nose_to_reward_port_sum,nose_to_reward_port_diff,thorax_velocity_sum,thorax_velocity_diff,to_reward_port_angle_sum,...,agent,cluster_timestamp,first_timestamp,recording,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,34242,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[4.873386, 4.9368286, 4.95431, 4.9935803, 5.11...","[3.630166, 3.7930658, 4.140675, 4.730201, 5.42...","[1.4684393, 1.92691, 2.386264, 3.032248, 3.868...","[15.642463, 15.843375, 16.048643, 16.248707, 1...","[5.4306073, 5.6331806, 5.7147374, 5.6083183, 5...","[5.032282, 4.8800473, 4.662217, 4.401958, 4.11...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,34442,4.3,After trial,nan,34042,34642
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,35241,"[35041, 35042, 35043, 35044, 35045, 35046, 350...","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[26.412424, 25.048779, 23.673517, 22.475124, 2...","[24.234488, 22.897377, 21.647272, 20.52266, 19...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,35439,4.3,Subj 2 Only,nan,35041,35639
2,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,36339,"[36139, 36140, 36141, 36142, 36143, 36144, 361...","[13.881315, 14.176304, 14.438505, 14.65004, 14...","[20.872814, 21.642649, 22.377028, 22.993326, 2...","[19.293379, 20.148054, 20.855904, 21.357841, 2...","[6.936893, 7.039064, 7.0063467, 6.898076, 6.69...","[5.934906, 6.0408955, 5.9854646, 5.8209095, 5....","[0.8359046, 0.79514295, 0.75987095, 0.7393834,...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,36538,4.3,Subj 2 blocking Subj 1,nan,36139,36738
3,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,37336,"[37136, 37137, 37138, 37139, 37140, 37141, 371...","[14.58712, 14.6317215, 14.698479, 14.7541, 14....","[4.5502543, 4.3937583, 4.5302463, 4.6787696, 4...","[2.9129052, 2.954475, 2.96101, 3.1235678, 3.26...","[4.4460034, 3.5049973, 2.642458, 1.9523432, 1....","[2.4809754, 1.6541765, 0.9319356, 0.37681082, ...","[3.5158348, 3.5706627, 3.5573924, 3.5392659, 3...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,37535,4.3,Subj 2 blocking Subj 1,nan,37136,37735
4,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,38834,"[38634, 38635, 38636, 38637, 38638, 38639, 386...","[16.687664, 16.676865, 16.725145, 16.772022, 1...","[22.407978, 22.735888, 22.8885, 22.941162, 22....","[20.59827, 20.763409, 20.856441, 20.875114, 20...","[11.273786, 9.234783, 7.4643965, 6.0377355, 5....","[9.619022, 7.865918, 6.3134904, 5.0081577, 4.0...","[1.2626256, 1.244121, 1.215351, 1.2018311, 1.1...",...,4.3,"[34079916, 34081302, 34082688, 34084074, 34084...",2067718,20240320_171038_alone_comp_subj_4-2_t6b6_merged,39033,4.3,Subj 2 Only,nan,38634,39233


In [80]:
comp_closeness_dict = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

In [81]:
grouped_exploded_columns["trial_label"] = grouped_exploded_columns.apply(lambda x: "win" if x["current_subject"] == x["condition "]  else ("lose" if x["agent"] == x["condition "] else "tie"), axis=1)
                                                                        

In [82]:
grouped_exploded_columns["competition_closeness"] = grouped_exploded_columns["competition_closeness"].map(comp_closeness_dict)

## Getting the ranges of each cluster

In [ ]:
grouped_exploded_columns["cluster_index_ranges_dict"] = grouped_exploded_columns["kmeans_cluster"].apply(lambda x: find_consecutive_ranges(x))

In [ ]:
grouped_exploded_columns.head()

In [ ]:
grouped_exploded_columns["cluster_times"] = grouped_exploded_columns.apply(lambda x: (np.array(x["cluster_timestamp"]) - x["first_timestamp"]) // 20, axis=1)


In [ ]:
grouped_exploded_columns["cluster_times_ranges_dict"] = grouped_exploded_columns.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_times"]), axis=1)

In [ ]:
grouped_exploded_columns["cluster_timestamps_ranges_dict"] = grouped_exploded_columns.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_timestamp"]), axis=1)

In [ ]:
grouped_exploded_columns["trial_cluster_times_ranges_dict"] = grouped_exploded_columns.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_times_ranges_dict"].items()}, axis=1)


In [ ]:
grouped_exploded_columns["trial_cluster_timestamps_ranges_dict"] = grouped_exploded_columns.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_timestamps_ranges_dict"].items()}, axis=1)


In [ ]:
# Define columns to be transformed into numpy arrays
array_columns = ['cluster_index_ranges_dict',
       'cluster_times_ranges_dict', 'cluster_timestamps_ranges_dict',
       'trial_cluster_timestamps_ranges_dict',
       'trial_cluster_times_ranges_dict']

first_value_columns = ['session_dir', 'sleap_name',
       'video_id', 'agent', 'first_timestamp', 'recording']

# Define aggregation dictionary
# agg_dict = {col: lambda x: np.array(x) for col in array_columns}
agg_dict = {col: list for col in array_columns}

agg_dict.update({col: 'first' for col in first_value_columns})

# Apply groupby and aggregation
video_grouped_exploded_columns = grouped_exploded_columns.groupby(["video_name", "current_subject"]).agg(agg_dict).reset_index()


In [ ]:
for col in array_columns:
    video_grouped_exploded_columns[col] = video_grouped_exploded_columns[col].apply(lambda x: combine_dicts(x))

In [ ]:
video_grouped_exploded_columns.head()

In [ ]:
video_grouped_exploded_columns.to_pickle("./proc/{}_cluster_ranges.pkl".format(OUTPUT_PREFIX))

In [ ]:
raise ValueError()

In [ ]:
1600609/1000

In [ ]:
merged_LFP_SPECTRAL_DF